## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN","NAME"],1)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df.CLASSIFICATION.value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1248        1
C1732        1
C2190        1
C4500        1
C2170        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts[class_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C2400        6
C4100        6
C1720        6
C1600        5
C1257        5
C2710        3
C1260        3
C0           3
C1234        2
C1267        2
C3200        2
C1256        2
C1246        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_with_dummies_df = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
X = application_with_dummies_df.drop(["IS_SUCCESSFUL"], axis='columns').values
y = application_with_dummies_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4400      
                                                                 
 dense_1 (Dense)             (None, 30)                3030      
                                                                 
 dense_2 (Dense)             (None, 10)                310       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 7,751
Trainable params: 7,751
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=120)

Epoch 1/120
804/804 [==============================] - 2s 3ms/step - loss: 0.5346 - accuracy: 0.7409
Epoch 2/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5344 - accuracy: 0.7411
Epoch 3/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5342 - accuracy: 0.7411
Epoch 4/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5343 - accuracy: 0.7410
Epoch 5/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5343 - accuracy: 0.7407
Epoch 6/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5341 - accuracy: 0.7407
Epoch 7/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5342 - accuracy: 0.7409
Epoch 8/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5342 - accuracy: 0.7412
Epoch 9/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5339 - accuracy: 0.7411
Epoch 10/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5339 - accura

804/804 [==============================] - 2s 2ms/step - loss: 0.5318 - accuracy: 0.7420
Epoch 82/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5322 - accuracy: 0.7416
Epoch 83/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5319 - accuracy: 0.7418
Epoch 84/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5318 - accuracy: 0.7420
Epoch 85/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5319 - accuracy: 0.7418
Epoch 86/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5318 - accuracy: 0.7420
Epoch 87/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5318 - accuracy: 0.7419
Epoch 88/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5322 - accuracy: 0.7423
Epoch 89/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5318 - accuracy: 0.7420
Epoch 90/120
804/804 [==============================] - 2s 2ms/step - loss: 0.5318 - accuracy: 

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5605 - accuracy: 0.7261 - 461ms/epoch - 2ms/step
Loss: 0.5605047941207886, Accuracy: 0.726064145565033


In [20]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
nn.save('AlphabetSoupCharity_optimized1.h5')


Overview of the analysis: 
Purpose of this analysis was to identify the model that provided the best accuracy.


Results: 


Data Preprocessing

What variable(s) are considered the target(s) for your model?
- Target = "Is-Successful" column

What variable(s) are considered to be the features for your model?
- NAME, 
- APPLICATION, 
- TYPE, 
- AFFILIATION, 
- CLASSIFICATION, 
- USE_CASE, 
- ORGANIZATION, 
- INCOME_AMT,
- SPECIAL_CONSIDERATIONS, 
- STATUS, and 
- ASK_AMT

What variable(s) are neither targets nor features, and should be removed from the input data?
- EIN


Compiling, Training, and Evaluating the Model

How many neurons, layers, and activation functions did you select for your neural network model, and why?
- 3 hidden layers
- Used relu and sigmoid

Were you able to achieve the target model performance?
- No

What steps did you take to try and increase model performance?
- Added a hidden layer and adjusted the # of epochs. Made a minor positive increase


Summary: 
- I was not able to attain an accuracy greater than 75%. Changes only improved score from 72.54% to 72.61%